# 1. AI plan

## 1.1 Basis functies

In [1]:
# Basis functies
# import numpy as np
# import pandas as pd
# from scipy.spatial import ConvexHull
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# import matplotlib.patches as mpatches

# # Zone definitie
# ZONES = {
#     'left_progression': {'x': [-17.5, 17.5], 'y': [-36, -12]},
#     'center_progression': {'x': [-17.5, 17.5], 'y': [-12, 12]},
#     'right_progression': {'x': [-17.5, 17.5], 'y': [12, 36]},
#     'left_final_third': {'x': [17.5, 52.5], 'y': [-36, -12]},
#     'center_final_third': {'x': [17.5, 52.5], 'y': [-12, 12]},
#     'right_final_third': {'x': [17.5, 52.5], 'y': [12, 36]},
# }

# def get_ball_zone(ball_x, ball_y, zones=ZONES):
#     """Bepaal in welke zone de bal zich bevindt"""
#     for zone_name, coords in zones.items():
#         x_min, x_max = coords['x']
#         y_min, y_max = coords['y']
        
#         if x_min <= ball_x <= x_max and y_min <= ball_y <= y_max:
#             return zone_name
    
#     return 'own_third'  # of None

# def calculate_convex_hull_area(points):
#     """Bereken oppervlakte van convex hull"""
#     if len(points) < 3:
#         return None
#     try:
#         hull = ConvexHull(points)
#         return hull.volume  # in 2D is dit de oppervlakte
#     except:
#         return None

# def calculate_team_compactness(team_df):
#     """
#     Bereken compactness metrics voor een team op een moment
    
#     Returns dict met:
#     - area: oppervlakte convex hull
#     - vertical_spread: verschil tussen hoogste en laagste speler
#     - horizontal_spread: verschil tussen meest linke en rechtse speler
#     """
#     if len(team_df) < 3:
#         return None
    
#     points = team_df[['x', 'y']].values
    
#     return {
#         'area': calculate_convex_hull_area(points),
#         'vertical_spread': team_df['x'].max() - team_df['x'].min(),
#         'horizontal_spread': team_df['y'].max() - team_df['y'].min(),
#         'mean_x': team_df['x'].mean(),
#         'mean_y': team_df['y'].mean(),
#         'defensive_line': team_df['x'].max(),  # hoogste (meest aanvallende) verdediger
#     }

# def get_ball_position(tracking_df, frame_id):
#     """Haal bal positie op voor een frame"""
#     ball_data = tracking_df[
#         (tracking_df['frame_id'] == frame_id) & 
#         (tracking_df['player_id'] == 'ball')  # of hoe jullie bal aanduiden
#     ]
    
#     if len(ball_data) == 0:
#         return None
    
#     return {
#         'x': ball_data['x'].iloc[0],
#         'y': ball_data['y'].iloc[0]
#     }

## 1.2 Analyse per balzone

In [2]:
# def analyze_block_by_ball_position(tracking_df, defending_team_id, 
#                                    possession_frames=None, zones=ZONES):
#     """
#     Analyseer verdedigend blok per balzone
    
#     Parameters:
#     - tracking_df: DataFrame met tracking data
#     - defending_team_id: team_id van verdedigende ploeg
#     - possession_frames: list van frame_ids waar aanvallende ploeg bal heeft
#     - zones: dict met zone definities
    
#     Returns:
#     - dict per zone met gemiddelde posities en compactness metrics
#     """
    
#     if possession_frames is None:
#         # Als niet meegegeven, neem alle frames (kan je later filteren)
#         possession_frames = tracking_df['frame_id'].unique()
    
#     results = {zone_name: {
#         'positions': [],
#         'compactness_metrics': [],
#         'frame_count': 0
#     } for zone_name in zones.keys()}
    
#     results['own_third'] = {
#         'positions': [],
#         'compactness_metrics': [],
#         'frame_count': 0
#     }
    
#     # Sample frames (niet elk frame analyseren = sneller)
#     sampled_frames = possession_frames[::10]  # elke 10e frame
    
#     for frame_id in sampled_frames:
#         # Haal bal positie op
#         ball_pos = get_ball_position(tracking_df, frame_id)
        
#         if ball_pos is None:
#             continue
        
#         # Bepaal zone
#         zone = get_ball_zone(ball_pos['x'], ball_pos['y'], zones)
        
#         # Haal verdedigers op
#         defenders = tracking_df[
#             (tracking_df['frame_id'] == frame_id) &
#             (tracking_df['team_id'] == defending_team_id)
#         ]
        
#         if len(defenders) < 3:
#             continue
        
#         # Bereken metrics
#         compactness = calculate_team_compactness(defenders)
        
#         if compactness is None:
#             continue
        
#         # Sla op
#         results[zone]['frame_count'] += 1
#         results[zone]['compactness_metrics'].append(compactness)
        
#         # Sla posities per speler op
#         for _, player in defenders.iterrows():
#             results[zone]['positions'].append({
#                 'player_id': player['player_id'],
#                 'x': player['x'],
#                 'y': player['y']
#             })
    
#     # Bereken gemiddelden per zone
#     summary = {}
    
#     for zone_name, data in results.items():
#         if data['frame_count'] == 0:
#             continue
        
#         # Gemiddelde posities per speler
#         positions_df = pd.DataFrame(data['positions'])
#         avg_positions = positions_df.groupby('player_id')[['x', 'y']].mean()
        
#         # Gemiddelde compactness metrics
#         metrics_df = pd.DataFrame(data['compactness_metrics'])
#         avg_metrics = metrics_df.mean().to_dict()
        
#         summary[zone_name] = {
#             'avg_positions': avg_positions,
#             'avg_compactness': avg_metrics,
#             'frame_count': data['frame_count']
#         }
    
#     return summary

## 1.3 Viz

In [3]:
# def plot_defensive_block_by_zone(analysis_results, zones=ZONES, 
#                                  save_path=None):
#     """
#     Visualiseer gemiddelde posities per balzone
#     """
    
#     # Filter zones met data
#     zones_with_data = {k: v for k, v in analysis_results.items() 
#                        if v['frame_count'] > 10}  # minimaal 10 frames
    
#     n_zones = len(zones_with_data)
    
#     if n_zones == 0:
#         print("Geen data om te visualiseren")
#         return
    
#     # Maak subplots
#     cols = 3
#     rows = (n_zones + cols - 1) // cols
    
#     fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
#     if n_zones == 1:
#         axes = [axes]
#     else:
#         axes = axes.flatten()
    
#     for idx, (zone_name, data) in enumerate(zones_with_data.items()):
#         ax = axes[idx]
        
#         # Teken veld
#         ax.set_xlim(-52.5, 52.5)
#         ax.set_ylim(-36, 36)
#         ax.set_aspect('equal')
        
#         # Middenlijn
#         ax.axvline(0, color='lightgray', linewidth=1, linestyle='--')
#         ax.axhline(0, color='lightgray', linewidth=1, linestyle='--')
        
#         # Teken zones (licht)
#         for zn, zc in zones.items():
#             rect = Rectangle(
#                 (zc['x'][0], zc['y'][0]),
#                 zc['x'][1] - zc['x'][0],
#                 zc['y'][1] - zc['y'][0],
#                 linewidth=1, edgecolor='lightgray',
#                 facecolor='lightgray', alpha=0.1
#             )
#             ax.add_patch(rect)
        
#         # Highlight huidige zone
#         if zone_name in zones:
#             zc = zones[zone_name]
#             rect = Rectangle(
#                 (zc['x'][0], zc['y'][0]),
#                 zc['x'][1] - zc['x'][0],
#                 zc['y'][1] - zc['y'][0],
#                 linewidth=2, edgecolor='red',
#                 facecolor='yellow', alpha=0.2
#             )
#             ax.add_patch(rect)
        
#         # Plot gemiddelde posities
#         positions = data['avg_positions']
        
#         ax.scatter(positions['x'], positions['y'], 
#                   s=200, c='blue', alpha=0.7, edgecolors='darkblue', linewidth=2)
        
#         # Voeg player IDs toe
#         for player_id, pos in positions.iterrows():
#             ax.text(pos['x'], pos['y'], str(player_id)[-2:],  # laatste 2 cijfers
#                    ha='center', va='center', fontsize=8, color='white', weight='bold')
        
#         # Convex hull
#         if len(positions) >= 3:
#             points = positions[['x', 'y']].values
#             hull = ConvexHull(points)
#             for simplex in hull.simplices:
#                 ax.plot(points[simplex, 0], points[simplex, 1], 'b-', alpha=0.3)
        
#         # Metrics in titel
#         metrics = data['avg_compactness']
#         title = f"{zone_name.replace('_', ' ').title()}\n"
#         title += f"Area: {metrics.get('area', 0):.0f}m² | "
#         title += f"V-spread: {metrics.get('vertical_spread', 0):.1f}m | "
#         title += f"H-spread: {metrics.get('horizontal_spread', 0):.1f}m\n"
#         title += f"Frames: {data['frame_count']}"
        
#         ax.set_title(title, fontsize=10, pad=10)
#         ax.set_xlabel('X (← own goal | opponent goal →)')
#         ax.set_ylabel('Y (← left | right →)')
        
#     # Verwijder lege subplots
#     for idx in range(n_zones, len(axes)):
#         fig.delaxes(axes[idx])
    
#     plt.tight_layout()
    
#     if save_path:
#         plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
#     plt.show()
    
#     return fig

## 1.5 Worklow

In [4]:
# # 1. Haal België possession frames op
# belgium_possession_frames = get_possession_frames(
#     tracking_df, 
#     events_df, 
#     belgium_team_id
# )

# # 2. Analyseer Macedonië's blok per balzone
# block_analysis = analyze_block_by_ball_position(
#     tracking_df=tracking_df,
#     defending_team_id=macedonia_team_id,
#     possession_frames=belgium_possession_frames,
#     zones=ZONES
# )

# # 3. Visualiseer
# plot_defensive_block_by_zone(block_analysis, save_path='outputs/block_by_zone.png')

# # 4. Print summary statistics
# for zone_name, data in block_analysis.items():
#     if data['frame_count'] > 0:
#         print(f"\n{zone_name}:")
#         print(f"  Frames analyzed: {data['frame_count']}")
#         print(f"  Average area: {data['avg_compactness']['area']:.1f} m²")
#         print(f"  Defensive line: {data['avg_compactness']['defensive_line']:.1f} m")
#         print(f"  Vertical spread: {data['avg_compactness']['vertical_spread']:.1f} m")

In [5]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from config import setup_logging
from extract import MatchDataLoader
from transform import transform_tracking_to_long_format, get_possession_frames

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Reload modules when code is changed (uncomment for development)
%load_ext autoreload
%autoreload 2

# Init logging
logger = setup_logging(log_file="../logs/dominance_analysis.log")

# 1. Load data

In [14]:
# Initialize data loader
loader = MatchDataLoader()

# Load all data
events_data, player_data, tracking_data, mapping_data = loader.load_all_data()

# Transform tracking data to long format
long_format_data = transform_tracking_to_long_format(tracking_data)

2025-11-10 20:59:36,948 - extract.data_loading - INFO - ✓ Loaded 1710 events from ../data/20251010-Belgium-North-Macedonia/events.json
2025-11-10 20:59:36,949 - extract.data_loading - INFO - ✓ Loaded 32 players from ../data/20251010-Belgium-North-Macedonia/events.json
2025-11-10 20:59:37,859 - extract.data_loading - INFO - ✓ Loaded 145567 tracking entries from ../data/20251010-Belgium-North-Macedonia/tracking.pkl
2025-11-10 20:59:37,861 - extract.data_loading - INFO - ✓ Loaded mapping data from ../data/20251010-Belgium-North-Macedonia/mapping.json


In [15]:
long_format_data.head()

,frame_id,period_id,timestamp,wall_clock,team_name,shirt_number,x,y,speed,last_touch
0,0,1,0.0,1760121954440,North Macedonia,10,2.36,-8.89,0.0,Belgium
1,0,1,0.0,1760121954440,North Macedonia,18,12.21,4.58,0.0,Belgium
2,0,1,0.0,1760121954440,North Macedonia,2,21.48,5.98,0.0,Belgium
3,0,1,0.0,1760121954440,North Macedonia,20,-0.08,9.08,0.0,Belgium
4,0,1,0.0,1760121954440,North Macedonia,21,15.56,-3.79,0.0,Belgium


In [ ]:
# Get possession frames for Belgium
belgium_possession_frames = get_possession_frames(
    long_format_data,
    "Belgium"
)

belgium_possession_frames[:10]

2025-11-10 21:01:52,173 - transform.team_possession_frames - INFO - Found 98303 unique possession frame ids for Belgium


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])